In [ ]:
from itertools import product

import torch
import plotly.io as pio
import plotly.express as px
from plotly import graph_objects as go
import pandas as pd
import numpy as np
import statsmodels.api as sm

from time_series_prediction import settings, utils

pio.templates.default = 'plotly_white'

In [ ]:
n_neurons = np.array([100, 200, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000])
n_neurons

In [ ]:
n_ensemble = 5

kwargs_default = {
    'spectral_radius': 0.99,
    'leaking_rate': 0.1,
    'k_l2': 1,
}

params = [
    ('n_neurons', n_neurons)
]
keys, vals = zip(*params)
param_dicts = []
for val in product(*vals):
    param_dicts.append({k: v for k, v in zip(keys, val)})
print(f'{len(param_dicts)} simulations')

In [ ]:
settings.switch_device('cpu')

sweep = utils.SweepESN(
    param_dicts,
    kwargs_default,
    n_ensemble,
    noise=2e-5,
    source='lorenz',
)

timing_list = []
for d in sweep.param_dicts:
    esn_kwargs = sweep.esn_kwargs | d
    t = sweep.run_ensemble(esn_kwargs, return_timing=True)[-1]
    timing_list.append(t)
    print(d['n_neurons'], np.mean(t, axis=0))

timing_arr_cpu = np.array(timing_list)

In [ ]:
settings.switch_device('cuda')

sweep_2 = utils.SweepESN(
    param_dicts,
    kwargs_default,
    n_ensemble,
    noise=2e-5,
    source='lorenz',
)

timing_list = []
for d in sweep_2.param_dicts:
    esn_kwargs = sweep_2.esn_kwargs | d
    t = sweep_2.run_ensemble(esn_kwargs, return_timing=True)[-1]
    timing_list.append(t)
    print(d['n_neurons'], np.mean(t, axis=0))

timing_arr_gpu = np.array(timing_list)

In [ ]:
timing_arr_cpu_mod = timing_arr_cpu.copy()
timing_arr_cpu_mod[..., 2] -= timing_arr_cpu_mod[..., 1]
timing_arr_cpu_mod[..., 1] -= timing_arr_cpu_mod[..., 0]

timing_arr_gpu_mod = timing_arr_gpu.copy()
timing_arr_gpu_mod[..., 2] -= timing_arr_gpu_mod[..., 1]
timing_arr_gpu_mod[..., 1] -= timing_arr_gpu_mod[..., 0]

In [ ]:
fig = go.Figure()
colorway = pio.templates['plotly_white'].layout.colorway
timing_info = [
    (1, 'generation', 'dash'),
    (0, 'training', 'dot'),
    (2, 'testing', None),
]
for i, name, dash in timing_info:
    fig.add_scatter(
        x=n_neurons,
        y=timing_arr_cpu_mod[..., i].mean(axis=1),
        line_color=colorway[0],
        line_dash=dash,
        error_y = {
            'type': 'data',
            'symmetric': False,
            'array': timing_arr_cpu_mod[..., i].max(axis=1) - timing_arr_cpu_mod[..., i].mean(axis=1),
            'arrayminus': timing_arr_cpu_mod[..., i].mean(axis=1) - timing_arr_cpu_mod[..., i].min(axis=1),
        },
        name=f'CPU, {name}',
    )
    fig.add_scatter(
        x=n_neurons,
        y=timing_arr_gpu_mod[..., i].mean(axis=1),
        line_color=colorway[1],
        line_dash=dash,
        error_y = {
            'type': 'data',
            'symmetric': False,
            'array': timing_arr_gpu_mod[..., i].max(axis=1) - timing_arr_gpu_mod[..., i].mean(axis=1),
            'arrayminus': timing_arr_gpu_mod[..., i].mean(axis=1) - timing_arr_gpu_mod[..., i].min(axis=1),
        },
        name=f'GPU, {name}',
    )
fig.update_xaxes(title_text='N')
fig.update_yaxes(title_text='Computation time (s)')
fig.update_layout(height=600)
fig

In [ ]:
y = timing_arr_cpu_mod[..., 0].mean(axis=1)
x = sm.add_constant(np.stack([n_neurons, n_neurons**2, n_neurons**3]).T)
mod = sm.OLS(y, x)
res = mod.fit()
res.summary()